# Variationist Example 1: Custom Metrics

**Welcome! In this example guide, you will be shown how to use your own custom metrics inside 🕵️‍♀️ Variationist.**


In [1]:
# Install using pip
!pip install variationist

Defaulting to user installation because normal site-packages is not writeable
  Obtaining dependency information for variationist from https://files.pythonhosted.org/packages/6b/44/611c77ef55974d3e0b65e6741d655567606b5226fc6475871d239cbdb742/variationist-0.1.3-py3-none-any.whl.metadata
  Obtaining dependency information for altair==5.2.0 from https://files.pythonhosted.org/packages/c5/e4/7fcceef127badbb0d644d730d992410e4f3799b295c9964a172f92a469c7/altair-5.2.0-py3-none-any.whl.metadata
  Obtaining dependency information for datasets==2.17.1 from https://files.pythonhosted.org/packages/73/75/ead8e4489217835fd8004c9c89dad8217deacd7e2591340dbe249b58c29f/datasets-2.17.1-py3-none-any.whl.metadata
  Obtaining dependency information for emoji==2.10.1 from https://files.pythonhosted.org/packages/98/00/00d56e704d69cee4a92b1d517676579b4af5f2f8bc72946c464a504705b2/emoji-2.10.1-py2.py3-none-any.whl.metadata
  Obtaining dependency information for geopandas==0.14.3 from https://files.pythonhosted.or

In [2]:
from variationist import Inspector, InspectorArgs, Visualizer, VisualizerArgs

/Users/camilla/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:34: NotOpenSSLWarning: urllib3 v2.0 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
/Users/camilla/Library/Python/3.9/lib/python/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
None of PyTorch, TensorFlow >= 2.0, or Flax have been found. Models won't be available and only tokenizers, configuration and file/data utilities can be used.


Load your own dataset in .tsv or .csv format, or use a dataset that is available on HuggingFace


In [3]:
# my_dataset = "my_dataset.tsv"
my_dataset = "hf::truthful_qa::generation::validation"

Let's use a metric of our own in addition to the built-in ones of Variationist. 

Here is a metric that counts how often questions contain at least one of the 5 W-words (what, where, who, why, when) in the questions of each category of the `truthful_qa` dataset.

In [9]:
def count_5w(label_values_dict, subsets_of_interest, inspector_args):
    results_dict = {}
    # for each variable we are analyzing
    for label in label_values_dict:
        results_dict[label] = {}
        # we loop through each value that variable can take
        for i in range(len(label_values_dict[label])):
            results_dict[label][label_values_dict[label][i]] = 0
            # we loop through each text in the subset we are currently analyzing
            for text in subsets_of_interest[label][i]:
                for w in ["what", "where", "who", "why", "when"]:
                    if w in text:
                        results_dict[label][label_values_dict[label][i]] += 1
                        
    return results_dict

In [15]:
# Define the inspector arguments. For now, let's just select a handful of metrics.
inspector_args = InspectorArgs(text_names=["question"],
                               var_names=["category"],
							   metrics=["pmi",
							            "stats",
									    "npw_relevance",
										"ttr",
										"freq",
          								count_5w],
							   language="en",
							   n_tokens=1,
							   n_cooc=1)

In [16]:
# Create an inspector instance, run it, and get the results in json
results = Inspector(dataset=my_dataset, args=inspector_args).inspect()

INFO: No values have been set for var_types. Defaults to nominal.
INFO: No values have been set for var_semantics. Defaults to general.
INFO: The metadata we will be using for the current analysis are:
{'text_names': ['question'], 'var_names': ['category'], 'metrics': ['pmi', 'stats', 'npw_relevance', 'ttr', 'freq', 'count_5w'], 'var_types': ['nominal'], 'var_semantics': ['general'], 'var_subsets': None, 'var_bins': [0], 'tokenizer': 'whitespace', 'language': 'en', 'n_tokens': 1, 'n_cooc': 1, 'unique_cooc': False, 'cooc_window_size': 0, 'freq_cutoff': 3, 'stopwords': False, 'custom_stopwords': None, 'lowercase': False, 'ignore_null_var': False}
INFO: all column identifiers are treated as column names.
INFO: 'Loading hf::truthful_qa::generation::validation' as a HuggingFace dataset. We assume the third element in the specified string is the subset ("generation") and the last is the split ("validation").
INFO: Tokenizing the question column...


100%|██████████| 817/817 [00:00<00:00, 42191.44it/s]


INFO: Currently calculating metric: 'pmi'


100%|██████████| 38/38 [00:00<00:00, 17592.00it/s]


INFO: Currently calculating metric: 'stats'
INFO: Currently calculating metric: 'npw_relevance'


...

INFO: Currently calculating metric: 'ttr'


100%|██████████| 38/38 [00:00<00:00, 9839.10it/s]


INFO: Currently calculating metric: 'freq'


100%|██████████| 38/38 [00:00<00:00, 19076.43it/s]

INFO: Currently calculating metric: 'count_5w'


In [17]:
# Define the visualizer arguments
visualizer_args = VisualizerArgs(
	output_folder="output", zoomable=True, ngrams=None, output_formats=["html", "png"])

# Create dynamic visualizations of the results
charts = Visualizer(input_json=results, args=visualizer_args).create()

Reading json data...
INFO: Creating a BarChart object for metric "pmi"...
INFO: Saving it to the filepath: "output/pmi/BarChart.html".
INFO: Saving it to the filepath: "output/pmi/BarChart.png".
INFO: Creating a BarChart object for metric "stats"...
INFO: Saving it to the filepath: "output/stats/StatsBarChart.html".
INFO: Saving it to the filepath: "output/stats/StatsBarChart.png".
INFO: Creating a BarChart object for metric "npw_relevance"...
INFO: Saving it to the filepath: "output/npw_relevance/BarChart.html".
INFO: Saving it to the filepath: "output/npw_relevance/BarChart.png".
INFO: Creating a BarChart object for metric "ttr"...
INFO: Saving it to the filepath: "output/ttr/DiversityBarChart.html".
INFO: Saving it to the filepath: "output/ttr/DiversityBarChart.png".
INFO: Creating a BarChart object for metric "freq"...
INFO: Saving it to the filepath: "output/freq/BarChart.html".
INFO: Saving it to the filepath: "output/freq/BarChart.png".
INFO: Creating a BarChart object for metri

In [18]:
charts["count_5w"]["BarChart"]

alt.Chart(...)